In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
import re

In [328]:
import numpy as np

## Wszystkie linki na stronie wikipedia.org

In [3]:
page = requests.get("https://pl.wikipedia.org/wiki/Zygmunt_III_Waza")

In [4]:
soup = BeautifulSoup(page.content,'html.parser')

In [5]:
links = soup.find_all('a',{'href': True})

In [6]:
adress = [link['href'] for link in links]
names = [link.string for link in links]

In [7]:
df = pd.DataFrame({'adress':adress,
                 'name':names})

In [8]:
df.head(10)

,adress,name
0,#mw-head,nawigacja
1,#p-search,szukaj
2,/wiki/Plik:Zygmunt_III_Waza.jpg,None
3,/wiki/Plik:Coat_of_arms_of_Vasa_kings_of_Polan...,None
4,/wiki/Plik:Sigismundus_III_Rex_-_signature.png,None
5,/wiki/Kr%C3%B3l_Polski,król Polski i wielki książę litewski
6,/wiki/Krak%C3%B3w,Kraków
7,/wiki/Stefan_Batory,Stefan Batory
8,/wiki/W%C5%82adys%C5%82aw_IV_Waza,Władysław IV Waza
9,/wiki/W%C5%82adcy_Szwecji,król Szwecji


## Informacje o filmie z http://www.filmweb.pl/film/Player+One-2018-719889

In [369]:
def get_info(path):
    page = requests.get(path)
    soup = BeautifulSoup(page.content,'html.parser')
    title = soup.find_all(class_='inline filmTitle')[0].a["title"]
    
    director = ''
    if len(soup.find_all('li',itemprop="director")) != 0: 
        director = soup.find_all('li',itemprop="director")[0].get_text()
    
    rating_nmb = np.nan
    sidebar = soup.find_all('div',class_="filmRateBox")
    if len(sidebar[0].find_all(itemprop="ratingValue"))!= 0:
        rating = sidebar[0].find_all(itemprop="ratingValue")[0].string.strip()
        rating_nmb = float(rating.replace(",","."))
    
    info_table = soup.find_all(class_="filmInfo")[0].find_all("tr")
    b_o = np.nan
    box_office_name = info_table[len(info_table)-1].find('th').get_text()
    if box_office_name == 'boxoffice:':
        box_office = info_table[len(info_table)-1].find('td').get_text()
        box_office = re.findall('\$([\d\s]+)',box_office)[0].strip().replace(' ','')
        b_o = int(box_office)
    
    date = ''   
    if np.isnan(b_o):
        date_name = info_table[len(info_table)-1].find('th').get_text()
        if date_name == 'premiera:':
            date_all = info_table[len(info_table)-1].find('td').get_text()
            date = re.findall('(.*?)\(',date_all.strip())[0].strip()
    else:
        date_name = info_table[len(info_table)-2].find('th').get_text()
        if date_name == 'premiera:':
            date_all = info_table[len(info_table)-2].find('td').get_text()
            date = re.findall('(.*?)\(',date_all.strip())[0].strip()
    
    
    
    return {'title':title,
           'director':director,
           'rating':rating_nmb,
           'box_office':b_o,
           'date':date}

## Informacje o bezpośrednim adresie na filmweb.pl na podstawie tytułu filmu

In [221]:
def get_adress(name):
    path_end=name.split()[0]
    for part in name.split()[1:]:
        path_end += '%20' + part
    path_filmweb = "http://www.filmweb.pl/films/search?q=" + path_end
    
    page = requests.get(path_filmweb)
    soup = BeautifulSoup(page.content,'html.parser')

    res_path=''
    
    if soup.find(class_="filmPreview__link") != None:
        res = soup.find(class_="filmPreview__link")["href"]
        res_path = "http://www.filmweb.pl" + res

    return res_path

## Ranking filmów z http://www.boxofficemojo.com/yearly/chart/?p=.htm&yr=2018

In [273]:
tables = pd.read_html("http://www.boxofficemojo.com/yearly/chart/?p=.htm&yr=2018")

In [274]:
table = tables[5]
table.head()

,0,1,2,3,4,5,6,7,8
0,Rank,Movie Title (click to view),Studio function MojoDrop1() { var number = doc...,FilterA24 AAE Abr. ADC Amazon Annapurna Argo. ...,Total Gross / Theaters,Opening / Theaters,Open,Close,NaN
1,FilterA24 AAE Abr. ADC Amazon Annapurna Argo. ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,Black Panther,BV,"$688,796,094",4084,"$202,003,951",4020,2/16,-
3,2,Avengers: Infinity War,BV,"$282,438,300",4474,"$257,698,183",4474,4/27,-
4,3,A Quiet Place,Par.,"$149,421,689",3808,"$50,203,562",3508,4/6,-


In [275]:
table = table.drop(1,axis=0)
table = table.drop(0,axis=1)
table = table.drop(8,axis=1)
table = table.drop(0,axis=0)

In [276]:
table.columns=['Title','Studio','Box office','Total gross','Opennig','Open','Close']

In [277]:
table = table.reset_index().drop("index",axis=1)

In [278]:
table 

,Title,Studio,Box office,Total gross,Opennig,Open,Close
0,Black Panther,BV,"$688,796,094",4084,"$202,003,951",4020,2/16
1,Avengers: Infinity War,BV,"$282,438,300",4474,"$257,698,183",4474,4/27
2,A Quiet Place,Par.,"$149,421,689",3808,"$50,203,562",3508,4/6
3,Ready Player One,WB,"$131,044,603",4234,"$41,764,050",4234,3/29
4,Peter Rabbit,Sony,"$114,575,572",3725,"$25,010,928",3725,2/9
5,Fifty Shades Freed,Uni.,"$100,407,760",3768,"$38,560,195",3768,2/9
6,A Wrinkle in Time,BV,"$94,977,901",3980,"$33,123,609",3980,3/9
7,I Can Only Imagine,RAtt.,"$81,176,294",2894,"$17,108,914",1629,3/16
8,Rampage (2018),WB (NL),"$78,527,760",4115,"$35,753,093",4101,4/13
9,Game Night,WB (NL),"$68,428,094",3502,"$17,005,332",3488,2/23


In [67]:
table["Box office"] = table["Box office"].str.replace(',','')

In [68]:
table["Box office"] = table["Box office"].str.replace('$','')

In [288]:
table

,Title,Studio,Box office,Total gross,Opennig,Open,Close
0,Black Panther,BV,"$688,796,094",4084,"$202,003,951",4020,2/16
1,Avengers: Infinity War,BV,"$282,438,300",4474,"$257,698,183",4474,4/27
2,A Quiet Place,Par.,"$149,421,689",3808,"$50,203,562",3508,4/6
3,Ready Player One,WB,"$131,044,603",4234,"$41,764,050",4234,3/29
4,Peter Rabbit,Sony,"$114,575,572",3725,"$25,010,928",3725,2/9
5,Fifty Shades Freed,Uni.,"$100,407,760",3768,"$38,560,195",3768,2/9
6,A Wrinkle in Time,BV,"$94,977,901",3980,"$33,123,609",3980,3/9
7,I Can Only Imagine,RAtt.,"$81,176,294",2894,"$17,108,914",1629,3/16
8,Rampage (2018),WB (NL),"$78,527,760",4115,"$35,753,093",4101,4/13
9,Game Night,WB (NL),"$68,428,094",3502,"$17,005,332",3488,2/23


In [374]:
def film_base(table):
    df = pd.DataFrame()
    
    for title,b_o in zip(table["Title"],table["Box office"]):
        #print(title)
        adress = get_adress(title)
        
        if adress !='':
            infos = get_info(adress)
            #print(infos)
            row = pd.DataFrame(pd.Series(infos)).transpose()
            df = df.append(row)
        
    df.reset_index()
    return df

In [375]:
base = film_base(table)

In [379]:
base = base.reset_index().drop("index",axis=1)

In [380]:
base

,box_office,date,director,rating,title
0,1331291387,14 lutego 2018,Ryan Coogler,7.1,Czarna Pantera
1,630000000,26 kwietnia 2018,Anthony Russo,8.6,Avengers: Wojna bez granic
2,235373301,13 kwietnia 2018,John Krasinski,7.2,Ciche miejsce
3,545183218,6 kwietnia 2018,Steven Spielberg,7.4,Player One
4,310609917,13 kwietnia 2018,Will Gluck,7.1,Piotruś Królik
5,368307760,9 lutego 2018,James Foley,5.7,Nowe oblicze Greya
6,124256336,7 marca 2018,Ava DuVernay,5.3,Pułapka czasu
7,NaN,16 marca 2018,Andrew Erwin,7,I Can Only Imagine
8,334630557,11 maja 2018,Brad Peyton,6.2,Rampage: Dzika furia
9,108705293,9 marca 2018,John Francis Daley,7.1,Wieczór gier


In [382]:
base.sort_values("rating",ascending=False)

,box_office,date,director,rating,title
70,NaN,,,10,Itzhak Frey & Sohn
1,630000000,26 kwietnia 2018,Anthony Russo,8.6,Avengers: Wojna bez granic
27,39560753,20 kwietnia 2018,Wes Anderson,8.1,Wyspa psów
32,18386550,4 maja 2018,Andrew Hyatt,7.9,"Paweł, apostoł Chrystusa"
19,53267098,15 czerwca 2018,Greg Berlanti,7.8,Twój Simon
80,NaN,12 marca 2017,Aaron Katz,7.8,Bliźnięta
76,14631377,26 marca 2010,Sherry Hormann,7.7,Kwiat pustyni
67,NaN,2 lutego 2018,Andriej Zwiagincew,7.6,Niemiłość
62,NaN,15 listopada 2012,Matúš Vizár,7.6,Pandy
82,NaN,15 stycznia 2017,Christian Duguay,7.4,Gra w szklane kulki
